In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.model_selection import KFold
import joblib

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
Y = pd.read_csv('../data/Activity.csv',skipinitialspace=True, na_values='?')
Y.head()

In [ ]:
list_isotopes = Y.columns.values.tolist()
list_isotopes[0:5]

In [ ]:
# Để tải lại các bộ dữ liệu
loaded_data = np.load('../data/hpge-soil-gamma-41.npz')
x_train = loaded_data['x_train']
y_train = loaded_data['y_train']
x_test = loaded_data['x_test']
y_test = loaded_data['y_test']

print('x_train shape:\t', x_train.shape)
print('x_test shape:\t', x_test.shape)
print('y_train shape:\t', y_train.shape)
print('y_test shape:\t', y_test.shape)

In [ ]:
# compare scaling methods for mlp inputs on regression problem
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, PoissonRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge


# prepare dataset with input and output scalers, can be none
def get_scaled_dataset(input_scaler, output_scaler, x_train, y_train, x_test, y_test, 
                       input_scaler_path='scaler/input_scaler_ann.sav'):
    x_train = x_train
    x_test = x_test
    y_test = y_test
    y_train = y_train

    if input_scaler is not None:
        # fit scaler
        input_scaler.fit(x_train)
        # transform training dataset
        x_train = input_scaler.transform(x_train)
        # transform test dataset
        x_test = input_scaler.transform(x_test)
        # save the scaler
        # joblib.dump(input_scaler, input_scaler_path)
    
    if output_scaler is not None:
        # fit scaler
        output_scaler.fit(y_train)
        # transform training dataset
        y_train = output_scaler.transform(y_train)
        # transform test dataset
        y_test = output_scaler.transform(y_test)
        # save the scaler
        #joblib.dump(output_scaler, output_scaler_path)

    return x_train, y_train, x_test, y_test, output_scaler
    
def percentage_loss(y_true, y_pred):
    return ((y_pred - y_true) / y_true) * 100

def custom_loss(true_values, predicted_values):
    errors = []
    for true_val, pred_val in zip(true_values, predicted_values):
        loss = percentage_loss(true_val, pred_val)
        errors.append(loss)
    
    return np.mean(np.abs(errors)), errors

def thong_ke_gia_tri(danh_sach, nguong):
    if not danh_sach:
        return None

    # Tính giá trị nhỏ nhất, lớn nhất và trung bình
    min_value = min(danh_sach)
    max_value = max(danh_sach)
    average = sum(danh_sach) / len(danh_sach)
    
    count = 0
    indices = []
    for i, gia_tri in enumerate(danh_sach):
        if gia_tri > nguong:
            count += 1
            indices.append(i)
    
    return min_value, max_value, average, count, indices

def eval_results(y_pred, testy):
    ab_errors = []
    good_count = 0
    for i in range(np.shape(testy)[0]):
        ab_error = abs((testy[i]-y_pred[i])/testy[i]).tolist()[0]
        if ab_error <= 0.15:
            good_count = good_count +1
        ab_errors.append(ab_error)
    return   good_count/np.shape(testy)[0], ab_errors

In [ ]:
trainX, trainy, testX, testy, output_scaler = get_scaled_dataset(MinMaxScaler(), None, x_train, y_train, x_test, y_test)
input_shape = trainX.shape[1]
print(input_shape)

In [ ]:
train = []
test = []
for i in range(len(trainX)):
    train.append([trainX[i],trainy[i]])

for i in range(len(testX)):
    test.append([testX[i],testy[i]])

trainDataSet = torch.utils.data.DataLoader(train, batch_size=32)
testDataSet = torch.utils.data.DataLoader(test, batch_size=16)
x_train_torch = torch.tensor(trainX,device=device)
y_train_torch = torch.tensor(trainy,device=device)
x_test_torch = torch.tensor(testX,device=device)
y_test_torch = torch.tensor(testy,device=device)

In [ ]:
import torch
import torch.nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import ParameterSampler, ParameterGrid
from sklearn.metrics import mean_squared_error

trainDataSet = torch.utils.data.DataLoader(train, batch_size=32)
testDataSet = torch.utils.data.DataLoader(test, batch_size=16)
x_train_torch = torch.tensor(trainX,device=device)
y_train_torch = torch.tensor(trainy,device=device)
x_test_torch = torch.tensor(testX,device=device)
y_test_torch = torch.tensor(testy,device=device)

class Net(nn.Module):
    def __init__(self, input_shape, hiddenSize1, hiddenSize2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_shape, hiddenSize1)
        self.fc2 = nn.Linear(hiddenSize1, hiddenSize2)
        self.fc3 = nn.Linear(hiddenSize2, 41)  # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def evaluate(self, dataset):
        self.eval()
        total_loss = 0
        criterion = nn.MSELoss()
        with torch.no_grad():
            for x, y in dataset:
                x, y = x.to(device), y.to(device)
                output = self(x.float())
                loss = criterion(output, y.float())
                total_loss += loss.item()
        return total_loss / len(dataset)

    def train_model(self, trainDataset, valDataset, optimizer, criterion, num_epochs, model_path, best_loss):
        train_losses = []
        val_losses = []
        best_val_loss = best_loss
        for epoch in range(num_epochs):
            self.train()
            running_loss = 0.0
            for inputs, labels in trainDataset:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self(inputs.float())
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            avg_train_loss = running_loss / len(trainDataset)
            val_loss = self.evaluate(valDataset)
            train_losses.append(avg_train_loss)
            val_losses.append(val_loss)
            
            # Save the model with the lowest validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self, model_path)
                print(f"Best validation loss: {best_val_loss:.4f}")
        return val_losses, train_losses


param_grid = {
    'hiddenSize1': [32, 64, 128],
    'hiddenSize2': [32, 64, 128],
    'lr': [0.001, 0.005, 0.01, 0.02],
    'num_epochs': [200]
}
param_list = list(ParameterGrid(param_grid))
input_shape = trainX.shape[1]
best_loss = float('inf')
best_params = {}
model_path = "models/best_mlp.pth"

results = []
best_params = {}
for params in param_list:
    print(f"Training at params: {params}")
    model = Net(input_shape, params['hiddenSize1'], params['hiddenSize2']).to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])
    val_losses, train_losses = model.train_model(trainDataSet, testDataSet, optimizer, criterion, params['num_epochs'], model_path, best_loss)
    
    # Evaluate the final model on the validation set
    val_loss = min(val_losses)
    if val_loss < best_loss:
        best_loss = val_loss
        best_params = params
        
        print(f"Best params: {best_params}")
        print(f"Best validation loss: {best_loss:.4f}")

In [ ]:
print(f"Best params: {best_params}")
print(f"Best validation loss: {best_loss:.4f}")

In [ ]:
model_path = 'models/best_mlp.pth'
model = torch.load(model_path)
model.eval()

In [ ]:
outputs = model(x_test_torch.float())
val_pre = outputs.cpu().detach().numpy()
testy = y_test_torch.cpu().detach().numpy()
MAE = mean_absolute_error(val_pre,testy)
print("MAE: ",MAE)
MSE = mean_squared_error(val_pre,testy)
print("MSE: ",MSE)
RMSE = np.sqrt(mean_squared_error(val_pre,testy))
print("RMSE: ",RMSE)
#Root Mean Squared Log Error(RMSLE)
RMSLE = np.log(np.sqrt(mean_squared_error(val_pre,testy)))
print("RMSLE: ",RMSLE)
#R Squared (R2)
R2_Score = r2_score(testy, val_pre)
print("R2_Score: ",R2_Score)
mape = mean_absolute_percentage_error(testy, val_pre)
print("MAPE: ", mape)

In [ ]:
idxs = np.arange(np.shape(testy)[0])

k = 300
index = 2
plt.figure(figsize=(10, 5))
plt.plot(idxs[0:k], val_pre[0:k, index], label='Ground Truth')
plt.plot(idxs[0:k], testy[0:k, index], label='Predicted',  linestyle='--', color = "red")
plt.legend(loc='lower right')
plt.xlabel("Output index")
plt.ylabel("Output value")
plt.show()

In [ ]:
plt.scatter(val_pre[0:k, index], testy[0:k, index])
plt.show()

In [ ]:
def calculate_metrics(y_true, y_pred):
    epsilon = 1e-6  # Để tránh chia cho 0
    # R2 Score
    r2 = r2_score(y_true, y_pred)
    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    # MSE
    mse = mean_squared_error(y_true, y_pred)
    # RMSE
    rmse = np.sqrt(mse)
    # RMSLE
    rmsle = np.log(rmse)
    # MAPE
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon)))
    # Count the number of samples with percentage error < 0.15
    percentage_errors = np.abs((y_true - y_pred) / (y_true + epsilon))
    count_below_threshold = np.sum(percentage_errors < 0.15)
    percentage_below_threshold = count_below_threshold / len(y_true)
    
    return {
        "R2 Score": r2,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "RMSLE": rmsle,
        "MAPE": mape,
        "Count < 0.15": count_below_threshold,
        "Percentage Below Threshold": percentage_below_threshold
    }

In [ ]:
# Lưu kết quả đánh giá cho mỗi output
results = []
for i, output_name in enumerate(list_dongvi):
    testy_output = y_test[:, i]
    pre = val_pre[:, i]
    metrics = calculate_metrics(testy_output, pre)
    result = {
        "Output": output_name,
        **metrics
    }
    results.append(result)

df_results = pd.DataFrame(results)
df_results.to_csv("Results/mlp_evaluation_results.csv", encoding='utf-8', index=False)
print("Kết quả đánh giá đã được lưu vào mlp_evaluation_results_canbang.csv")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Deep CNN

In [ ]:
x_train_torch = torch.tensor(trainX, dtype=torch.float32).unsqueeze(1) 
y_train_torch = torch.tensor(trainy)
x_test_torch = torch.tensor(testX, dtype=torch.float32).unsqueeze(1)
y_test_torch = torch.tensor(testy)

train_dataset = TensorDataset(x_train_torch, y_train_torch)
test_dataset = TensorDataset(x_test_torch, y_test_torch)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import ParameterGrid

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Deep_RegressionCNN1D(nn.Module):
    def __init__(self, input_length, kernel_size, hiddenSize1, hiddenSize2, unit1, unit2):
        super(Deep_RegressionCNN1D, self).__init__()
        
        self.kernel_size = kernel_size
        
        # Block 1
        self.conv1_1 = nn.Conv1d(in_channels=1, out_channels=hiddenSize1, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv1d(in_channels=hiddenSize1, out_channels=hiddenSize1, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=kernel_size, stride=2, padding=0)
        
        # Block 2
        self.conv2_1 = nn.Conv1d(in_channels=hiddenSize1, out_channels=hiddenSize2, kernel_size=3, stride=1, padding=1)
        self.conv2_2 = nn.Conv1d(in_channels=hiddenSize2, out_channels=hiddenSize2, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=kernel_size, stride=2, padding=0)
        
        # Block 3
        self.conv3_1 = nn.Conv1d(in_channels=hiddenSize2, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=kernel_size, stride=2, padding=0)
        
        # Compute conv_output_size
        conv_output_size = self.compute_conv_output_size(input_length)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * conv_output_size, unit1)  # Adjusted for the new conv output size
        self.fc2 = nn.Linear(unit1, unit2)
        self.fc3 = nn.Linear(unit2, 41)
    
    def compute_conv_output_size(self, length):
        for _ in range(3):  # 3 pooling layers
            length = (length - self.kernel_size) // 2 + 1
        return length
    
    def forward(self, x):
        # Block 1
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        
        # Block 2
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        
        # Block 3
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.pool3(x)
        
        # Flatten and fully connected layers
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

    def train_model(self, trainDataset, valDataset, optimizer, criterion, num_epochs, model_path, best_loss):
        train_losses = []
        val_losses = []
        best_val_loss = best_loss
        for epoch in range(num_epochs):
            self.train()
            running_loss = 0.0
            for inputs, labels in trainDataset:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self(inputs.float())
                loss = criterion(outputs, labels.float())
                # loss = improved_custom_log_loss(labels.float().view(-1, 1), outputs)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            avg_train_loss = running_loss / len(trainDataset)
            val_loss = self.evaluate(valDataset)
            train_losses.append(avg_train_loss)
            val_losses.append(val_loss)
            
            # Save the model with the lowest validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self, model_path)
                print(f"Best validation loss: {best_val_loss:.4f}")
        return val_losses, train_losses

    def evaluate(self, valDataset):
        self.eval()
        val_loss = 0.0
        criterion = nn.MSELoss()
        with torch.no_grad():
            for inputs, labels in valDataset:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self(inputs.float())
                loss = criterion(outputs, labels.float())
                # loss = improved_custom_log_loss(outputs, labels.float().view(-1, 1))
                val_loss += loss.item()
        return val_loss / len(valDataset)

# Grid search parameters
param_grid = {
    'hiddenSize1': [32, 64],
    'hiddenSize2': [32, 64],
    'lr': [0.001, 0.005, 0.01],
    'num_epochs': [200],
    'kernel_size': [2, 3, 4],
    'unit1': [64, 128],
    'unit2': [64, 128],
}

# Generate all combinations of parameters from the grid
param_list = list(ParameterGrid(param_grid))

input_length = trainX.shape[1]
best_loss = float('inf')
best_params = {}
model_path = "models/best_cnn.pth"

for params in param_list:
    print(f"Training at params: {params}")
    model = Deep_RegressionCNN1D(input_length=input_length, kernel_size=params['kernel_size'], 
                                 hiddenSize1=params['hiddenSize1'], hiddenSize2=params['hiddenSize2'],
                                 unit1=params['unit1'], unit2=params['unit2']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])
    val_losses, train_losses = model.train_model(train_loader, test_loader, optimizer, criterion, params['num_epochs'], model_path, best_loss)
    
    # Evaluate the final model on the validation set
    val_loss = min(val_losses)
    if val_loss < best_loss:
        best_loss = val_loss
        best_params = params
        
        print(f"Best params: {best_params}")
        print(f"Best validation loss: {best_loss:.4f}")

In [ ]:
print(best_params)
print(best_loss)

In [ ]:
hiddenSize1 = best_params["hiddenSize1"]
hiddenSize2 = best_params["hiddenSize2"]
kernel_size = best_params["kernel_size"]

model_path = "models/best_cnn.pth"
model = torch.load(model_path)
model.eval()

In [ ]:
outputs = model(x_test_torch.to(device))
val_pre = outputs.cpu().detach().numpy()
testy = y_test_torch.cpu().detach().numpy()
MAE = mean_absolute_error(val_pre,testy)
print("MAE: ",MAE)
MSE = mean_squared_error(val_pre,testy)
print("MSE: ",MSE)
RMSE = np.sqrt(mean_squared_error(val_pre,testy))
print("RMSE: ",RMSE)
#Root Mean Squared Log Error(RMSLE)
RMSLE = np.log(np.sqrt(mean_squared_error(val_pre,testy)))
print("RMSLE: ",RMSLE)
#R Squared (R2)
R2_Score = r2_score(testy, val_pre)
print("R2_Score: ",R2_Score)
mape = mean_absolute_percentage_error(testy, val_pre)
print("MAPE: ", mape)

In [ ]:
idxs = np.arange(np.shape(testy)[0])

k = 300
index = 2
plt.figure(figsize=(10, 5))
plt.plot(idxs[0:k], val_pre[0:k, index], label='Ground Truth')
plt.plot(idxs[0:k], testy[0:k, index], label='Predicted',  linestyle='--', color = "red")
plt.legend(loc='lower right')
plt.xlabel("Output index")
plt.ylabel("Output value")
plt.show()

In [ ]:
plt.scatter(val_pre[0:k, index], testy[0:k, index])
plt.show()